- Average Annual Household Transportation Costs: Out-of-pocket expenses including tolls, parking, and transit fares

    - These costs do not include other vehicle operating and purchase costs.

    - Costs are annualized using factor of 320


In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '${:0,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# person data
person = pd.read_csv(output_path / 'agg/dash/person_cost.csv')
# household data
hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')

annual_factor = summary_config['weekday_to_annual']
# list of equity geographies
equity_geogs = summary_config['equity_geogs']
not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
df_person = person.copy()
df_person['is_rgc'] = "In RGC"
df_person.loc[df_person['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"
# included modes
# TODO: ask what this means
df_person = df_person[(df_person['mode'].isin(['SOV','HOV2','HOV3+']) & (df_person['dorp'] == 1)) 
                      | (~df_person['mode'].isin(['SOV','HOV2','HOV3+']))]

df_hh = hh.copy()
df_hh['is_rgc'] = "In RGC"
df_hh.loc[df_hh['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"

In [5]:
def annual_costs(geog, map=False):
    df = df_person.groupby(geog, as_index=False)['travcost_wt'].sum().set_index(geog)
    df['hhexpfac'] = df_hh.groupby(geog)['hhexpfac'].sum()

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )

    df.loc['Region'] = [df_person['travcost_wt'].sum(), df_hh['hhexpfac'].sum()]
    df['Average Annual Costs'] = (df['travcost_wt'] / df['hhexpfac']) * annual_factor




    return df[['Average Annual Costs']]

In [6]:
df = annual_costs('hh_county')
df = df[df.index!='Outside Region']
df

,Average Annual Costs
hh_county,
King,"$2,383"
Kitsap,"$2,837"
Pierce,"$2,783"
Snohomish,"$2,953"
Region,"$2,597"


In [7]:
annual_costs('is_rgc')

,Average Annual Costs
is_rgc,
In RGC,$954
Not in RGC,"$2,772"
Region,"$2,597"


In [8]:
annual_costs('hh_rgc')

,Average Annual Costs
hh_rgc,
Auburn,"$1,868"
Bellevue,$985
Bothell Canyon Park,"$2,391"
Bremerton,"$1,164"
Burien,"$1,815"
Everett,"$1,096"
Federal Way,"$1,947"
Greater Downtown Kirkland,"$1,824"
Kent,"$1,682"


In [9]:
annual_costs('hh_rg_proposed')

,Average Annual Costs
hh_rg_proposed,
Cities and Towns,"$3,399"
Core,"$2,528"
HCT,"$2,751"
Metro,"$1,689"
Rural,"$4,387"
Urban Unincorporated,"$3,187"
Region,"$2,597"


In [10]:
df = pd.DataFrame()
for name, col in {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
}.items():
    _df = annual_costs(col, map=True)
    _df['Group'] = name
    df = pd.concat([df, _df])

df = df.reset_index()
df
df.rename(columns={'index':'EFA Type'}, inplace=True)

df[['Group', 'EFA Type', 'Average Annual Costs']]


,Group,EFA Type,Average Annual Costs
0,People of Color,Below Regional Average,"$2,811"
1,People of Color,Above Regional Average,"$2,336"
2,People of Color,Higher Share of Equity Population,"$2,354"
3,People of Color,Region,"$2,597"
4,Income,Below Regional Average,"$2,806"
5,Income,Above Regional Average,"$2,410"
6,Income,Higher Share of Equity Population,"$2,062"
7,Income,Region,"$2,597"
8,LEP,Below Regional Average,"$2,653"
9,LEP,Above Regional Average,"$2,540"
